In [18]:
import sys
import os

# Go up to the `app/` folder
sys.path.append(os.path.abspath(".."))

from db.database import Database

In [19]:
ls

admin_page.py          login_page.py            test_import.py
bert_score_test.ipynb  old_draft_admin_page.py  user_page.py
draft_admin_page.py    __pycache__/
__init__.py            skeleton_admin_page.py


In [20]:
ls ..

db/  image.py  __init__.py  layouts/  main.py  prompt.py


In [21]:
ls ../db

create.py    example.ipynb  __init__.py  __pycache__/
database.py  images/        mma.db


In [22]:
dataset_path = os.path.abspath(os.path.join("..", "db", "mma.db"))

In [23]:
def get_mode(row):
    if row['enhanced_prompt'] and row['used_suggestion']:
        return 'Both'
    elif row['enhanced_prompt']:
        return 'Enhancement Only'
    elif row['used_suggestion']:
        return 'Suggestion Only'
    return 'Baseline'

def filter_by_mode(df, mode):
    if mode == 'overview':
        return df

    df['interaction_mode'] = df.apply(get_mode, axis=1)
    return df[df['interaction_mode'] == mode]

In [24]:
with Database(dataset_path) as db:
        df_images = db.fetch_all_images()
        df_prompts = db.fetch_all_prompts()
        df = df_images.merge(df_prompts, left_on="prompt_id", right_on="id", suffixes=("_image", "_prompt"))
        print(df)

   id_image  prompt_id  user_id_image  chat_id_image  prompt_guidance  \
0         1          1              1              1             7.00   
1         2          2              1              1             3.50   
2         3          3              1              2             5.75   
3         4          3              2              2             9.00   
4         5          4              3              3             4.00   
5         6          5              3              3             5.00   
6         7          6              3              3             6.00   

   image_guidance                         path  id_prompt  chat_id_prompt  \
0           0.375      ./images/test_image.jpg          1               1   
1           0.500     ./images/test_image2.jpg          2               1   
2           0.300     ./images/test_image3.jpg          3               2   
3           0.150     ./images/test_image4.jpg          3               2   
4           0.250         ./im

In [25]:
mode = "Both"
df_filtered = filter_by_mode(df, mode)

In [26]:
print(df_filtered)

   id_image  prompt_id  user_id_image  chat_id_image  prompt_guidance  \
1         2          2              1              1              3.5   
6         7          6              3              3              6.0   

   image_guidance                         path  id_prompt  chat_id_prompt  \
1            0.50     ./images/test_image2.jpg          2               1   
6            0.35  ./images/sauvage_sunset.png          6               3   

   user_id_prompt               prompt  depth  used_suggestion  \
1               1  another test prompt      2                1   
6               3   with sunset behind      2                1   

   modified_suggestion  suggestion_used  is_enhanced  \
1                    0  suggestion text            1   
6                    1           sunset            1   

                        enhanced_prompt image_in_id images_out  \
1                      enhanced version        None       None   
6  sauvage perfume with sunset ambiance        N

In [27]:
with Database(dataset_path) as db:  # adjust path if needed
    print("=== USERS ===")
    print(db.fetch_all_users())

    print("\n=== CHATS ===")
    print(db.fetch_all_chats())

    print("\n=== PROMPTS ===")
    print(db.fetch_all_prompts())

    print("\n=== IMAGES ===")
    print(db.fetch_all_images())


=== USERS ===
   id          username     password
0   1   first_test_user  password123
1   2  second_test_user  password456
2   3         fake_user     fakepass
3   4             alice  password123

=== CHATS ===
    id              title  user_id
0    1    First Test Chat        1
1    2   Second Test Chat        2
2    3  Sauvage Test Chat        3
3    4    First Test Chat        1
4    5   Second Test Chat        2
5    6  Sauvage Test Chat        3
6    7    First Test Chat        1
7    8   Second Test Chat        2
8    9  Sauvage Test Chat        3
9   10    First Test Chat        1
10  11   Second Test Chat        2
11  12  Sauvage Test Chat        3

=== PROMPTS ===
    id  chat_id  user_id                   prompt  depth  used_suggestion  \
0    1        1        1            a test prompt      1                0   
1    2        1        1      another test prompt      2                1   
2    3        2        2  yet another test prompt      1                0   
3    4

In [28]:
with Database(dataset_path) as db:
    df_images = db.fetch_all_images()
    df_prompts = db.fetch_all_prompts()

    df = df_images.merge(
        df_prompts,
        left_on="prompt_id",
        right_on="id",
        suffixes=("_image", "_prompt")
    )

    # Drop redundant columns where values are always equal
    for col in ["user_id", "chat_id", "id"]:
        col_image = f"{col}_image"
        col_prompt = f"{col}_prompt"
        if col_image in df.columns and col_prompt in df.columns:
            if df[col_image].equals(df[col_prompt]):
                df.drop(columns=[col_prompt], inplace=True)
                df.rename(columns={col_image: col}, inplace=True)

    # Rename merged `id` from prompt table if kept
    if 'id' not in df.columns and 'id_image' in df.columns:
        df.rename(columns={'id_image': 'id'}, inplace=True)

print(df)

   id  prompt_id  user_id_image  chat_id  prompt_guidance  image_guidance  \
0   1          1              1        1             7.00           0.375   
1   2          2              1        1             3.50           0.500   
2   3          3              1        2             5.75           0.300   
3   4          3              2        2             9.00           0.150   
4   5          4              3        3             4.00           0.250   
5   6          5              3        3             5.00           0.300   
6   7          6              3        3             6.00           0.350   

                          path  id_prompt  user_id_prompt  \
0      ./images/test_image.jpg          1               1   
1     ./images/test_image2.jpg          2               1   
2     ./images/test_image3.jpg          3               2   
3     ./images/test_image4.jpg          3               2   
4         ./images/sauvage.png          4               3   
5  ./images/sauva

In [29]:
with Database(dataset_path) as db:
    df_images = db.fetch_all_images()
    df_prompts = db.fetch_all_prompts()

# Merge on prompt_id
df = df_images.merge(
    df_prompts,
    left_on="prompt_id",
    right_on="id",
    suffixes=("_image", "_prompt")
)

# Rename prompt ID to keep clarity
df.rename(columns={"id_image": "image_id", "id_prompt": "prompt_id"}, inplace=True)

# Drop duplicate columns if values match
for col in ['user_id', 'chat_id']:
    col_img = f"{col}_image"
    col_prm = f"{col}_prompt"
    if col_img in df.columns and col_prm in df.columns:
        if df[col_img].equals(df[col_prm]):
            df.drop(columns=[col_prm], inplace=True)
            df.rename(columns={col_img: col}, inplace=True)

# Now you have a clean DataFrame
print(df.columns)
print(df[:7])

Index(['image_id', 'prompt_id', 'user_id_image', 'chat_id', 'prompt_guidance',
       'image_guidance', 'path', 'prompt_id', 'user_id_prompt', 'prompt',
       'depth', 'used_suggestion', 'modified_suggestion', 'suggestion_used',
       'is_enhanced', 'enhanced_prompt', 'image_in_id', 'images_out'],
      dtype='object')
   image_id  prompt_id  user_id_image  chat_id  prompt_guidance  \
0         1          1              1        1             7.00   
1         2          2              1        1             3.50   
2         3          3              1        2             5.75   
3         4          3              2        2             9.00   
4         5          4              3        3             4.00   
5         6          5              3        3             5.00   
6         7          6              3        3             6.00   

   image_guidance                         path  prompt_id  user_id_prompt  \
0           0.375      ./images/test_image.jpg          1      